In [ ]:
## Hi all..I am very new to Kaggle any suggestions would be very helpful
## In my this version of notebook I am focussed on Exploratory Data Analysis and Feature Engineering
## The remaining part that deals with  Predictive Modelling will upload in upcoming Versions*

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics 
from sklearn.metrics import confusion_matrix,roc_auc_score

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Getting the Data

In [ ]:
train_data = pd.read_csv("/kaggle/input/train.csv")
test_data  = pd.read_csv("/kaggle/input/test.csv")

# Exploratory Data Analysis

In [ ]:
train_data.shape

In [ ]:
test_data.shape

In [ ]:
train_data.columns

**It can be seen that our train_data has 891 records with 11 attributes and one target as Survived.
The Data Definition of each attribute can be given below as

* PassengerId: Unique Id of a passenger
* survived:    Survival 
* pclass:    Ticket class     
* sex:    Sex     
* Age:    Age in years     
* sibsp:    # of siblings / spouses aboard the Titanic     
* parch:    # of parents / children aboard the Titanic     
* ticket:    Ticket number     
* fare:    Passenger fare     
* cabin:    Cabin number     
* embarked:    Port of Embarkation


In [ ]:
train_data.head()

In [ ]:
train_data.describe(include = 'all')

In [ ]:
train_data.info()

Here it can be seen that 2 vars Age and Fare are float,5 vars are integers and other 5 are objects.
Also it can be seen that PassengerId,Ticket and Name are not that significant enough in predicting the Dependent var Survived.
Also most of the features needs to be converted before modelling

In [ ]:
## Checking the NA values in the dataset
train_data.isnull().sum()

So age has 177 missing values,Cabin has 682 and Embarked has 2 missing values

In [ ]:
(687/891) *100 ## So Cabin has 77% of missing values So we can drop it for further analysis as most of the data is missing.

# Lets check the data distribution through plots

In [ ]:
## Seaborn Relplot also gives good visualization
## A Relplot is a Figure-level interface for drawing relational plots

In [ ]:
sns.relplot(x="Sex", y="Age", hue="Survived", data=train_data)

In [ ]:
## Rechecking using statistics
x = train_data.Sex[train_data.Survived == 1]

In [ ]:
x.value_counts()

*  It can be seen that females have higher rate of survival
*  Also from the above relplot it can be seen that for males the rate of survival is better when the age is between 10-30
*  In both Male and female infants have good survival rate
*  For further analysis we need to create the age groups

In [ ]:
## We will plot Seaborn FacetGrid which is Multi-plot grid for plotting conditional relationships.

## We will plot it using vars Sex,Embark,Survived and PClass

In [ ]:
bins = np.arange(0, 65, 5)
g = sns.FacetGrid(train_data, row="Embarked")
g = g.map(sns.pointplot, 'Pclass', 'Survived','Sex', palette=None,  order=None, hue_order=None )
g.add_legend()

* From the above FacetGrid graph it can be seen that for Embarked 'S' and 'Q' females have higher rate of survival , whereas for Port 'C' male have higher rate of survival
The relation between PClass and Survived can be seen as

In [ ]:
sns.barplot(x='Pclass', y='Survived', data=train_data)

In [ ]:
##Checking the Class Distribution wrt Survived
x1 = train_data.Survived[train_data.Pclass == 3]
x2 = train_data.Survived[train_data.Pclass == 2]
x3 = train_data.Survived[train_data.Pclass == 1]


In [ ]:
x3.value_counts()

In [ ]:
x2.value_counts()

In [ ]:
x1.value_counts()

In [ ]:
## It can be seen from the above statistics that most of the people belonging to Class 3 did not survive
## It can also be rechecked by plotting FacetGrid as below ->

grid = sns.FacetGrid(train_data, col='Survived', row='Pclass', size=2.2, aspect=1.6)
grid.map(plt.hist, 'Age', alpha=.5, bins=20)
grid.add_legend();

In [ ]:
## It can be seen that Pclass 3 has very less rate of survival

In [ ]:
## Need to check relation between SibSp and Survived and relation between Parch and Survived
## We will check it by plotting the Seaborn Barplot
sns.barplot(y = 'Survived', x = 'SibSp',data = train_data, edgecolor = 'w')
plt.show()

sns.barplot(y = 'Survived', x = 'Parch',data = train_data, edgecolor = 'w')
plt.show()


In [ ]:
grid1 = sns.FacetGrid(train_data, col='Survived', row='SibSp', size=2.2, aspect=1.6)
grid1.map(plt.hist, 'Age', alpha=.5, bins=20)
grid1.add_legend();

In [ ]:
## It can be seen that the Survival is better for Sibsp = 0 whereas Sibsp with value 3,4,5 have least and SibSp with value as 8 no survival

In [ ]:
sns.relplot(x="SibSp", y="Parch", hue="Survived", data=train_data)

In [ ]:
grid2 = sns.FacetGrid(train_data, col='Survived', row='Parch', size=2.2, aspect=1.6)
grid2.map(plt.hist, 'Age', alpha=.5, bins=20)
grid2.add_legend();

# Feature Engineering

1. MISSING VALUE CONSTRAINT

In [ ]:
train_data.isnull().sum()

In [ ]:
## Age,Cabin, Embarked have missing values in train data

In [ ]:
test_data.isnull().sum()

In [ ]:
## Age,Fare,Cabin have missin values

In [ ]:
data = [train_data,test_data]

In [ ]:
train_data.Embarked.value_counts()

In [ ]:
## Most Commom value is 'S' so fillna with 'S'
train_data['Embarked'] = train_data['Embarked'].fillna('S')

In [ ]:
train_data.Embarked.value_counts()

In [ ]:
test_data.Fare.describe()

In [ ]:
## So we will fill the missing value with the mean
test_data['Fare'] = test_data['Fare'].fillna(35.627188)

In [ ]:
test_data.Fare.isnull().sum()

In [ ]:
train_data.Age.describe()

In [ ]:
## In both train_data and test_data Age attribute has missing values 
##We will fill na with the mean Age in both the data
mean_age = train_data.Age.mean()
mean_age
train_data['Age'] = train_data['Age'].fillna(mean_age)

In [ ]:
train_data.Age.isnull().sum()

In [ ]:
test_data.Age.describe()

In [ ]:
mean_age1 = test_data.Age.mean()
mean_age1
test_data['Age'] = test_data['Age'].fillna(mean_age1)

In [ ]:
test_data.Age.isnull().sum()

In [ ]:
## Now the attribute Cabin has the maximum nbr of missing values 
## In Cabin the letter refers to the Deck which might be helpful for Survived attr as in it might be the case 
## that people belonging to certain Deck have higher rate of survival

In [ ]:
train_data.Cabin.describe()

In [ ]:
train_data.Cabin.value_counts()

In [ ]:
train_data['Cabin'] = train_data['Cabin'].str.extract('(\w)')


In [ ]:
train_data.Cabin.value_counts()

In [ ]:
train_data.Cabin.describe()

In [ ]:
## Lets plot the relation between Cabin and Survived

In [ ]:
grid3 = sns.FacetGrid(train_data, col='Survived', row='Cabin', size=2.2, aspect=1.6)
grid3.map(plt.hist, 'Age', alpha=.5, bins=20)
grid3.add_legend();

In [ ]:
## It can be seen from the above Graph that Passengers with Deck no as A and F have higher rate for survival

In [ ]:
test_data.Cabin.describe()

In [ ]:
test_data['Cabin'] = test_data['Cabin'].str.extract('(\w)')

In [ ]:
test_data.Cabin.value_counts()

In [ ]:
test_data.Cabin.describe()

In [ ]:
## Lets fill na for Cabin in both the data with value as 'T'

In [ ]:
train_data['Cabin'] = train_data['Cabin'].fillna('T')

In [ ]:
test_data['Cabin'] = test_data['Cabin'].fillna('T')

In [ ]:
train_data.Cabin.value_counts()

In [ ]:
test_data.Cabin.value_counts()

# CONVERTING FEATURES INTO CORRECT FORMAT 

In [ ]:
train_data.describe(include = 'all')

In [ ]:
## Here the vars Sex,Cabin and Embarked need to be converted to numeric categorical
## So we will convert them using LabelEncoder object

In [ ]:
le = LabelEncoder()

train_data.Sex       = le.fit_transform(train_data.Sex)
train_data.Cabin     = le.fit_transform(train_data.Cabin)
train_data.Embarked  = le.fit_transform(train_data.Embarked)

train_data.Sex       = train_data.Sex.astype('category')
train_data.Cabin     = train_data.Cabin.astype('category')
train_data.Embarked  = train_data.Embarked.astype('category')

In [ ]:
train_data.describe(include = 'all')

In [ ]:
## Similarly we need to convert attr for test_data

In [ ]:
test_data.Sex       = le.fit_transform(test_data.Sex)
test_data.Cabin     = le.fit_transform(test_data.Cabin)
test_data.Embarked  = le.fit_transform(test_data.Embarked)

test_data.Sex       = test_data.Sex.astype('category')
test_data.Cabin     = test_data.Cabin.astype('category')
test_data.Embarked  = test_data.Embarked.astype('category')

In [ ]:
train_data.Pclass.value_counts()

In [ ]:
train_data.Pclass.describe()

In [ ]:
## Pclass needs to be categorized
train_data.Pclass = train_data.Pclass.astype('category')
test_data.Pclass  = test_data.Pclass.astype('category')

In [ ]:
test_data.Pclass.describe()

In [ ]:
##SibSp and Parch would make more sense as a combined feature, that shows the total number of relatives, a person has on the Titanic.
## I will create it below and also a feature that shows if someone is not alone.
data = [train_data, test_data]
for dataset in data:
    dataset['relatives'] = dataset['SibSp'] + dataset['Parch']
    dataset.loc[dataset['relatives'] > 0, 'not_alone'] = 1
    dataset.loc[dataset['relatives'] == 0, 'not_alone'] = 0

In [ ]:
train_data.columns

In [ ]:
train_data.relatives.head()

In [ ]:
train_data.Ticket.describe()
##Since the Ticket attribute has 681 unique tickets, it will be a bit tricky to convert them into useful categories. So we will drop it from the dataset.

In [ ]:
train_data.drop('Ticket', axis = 1, inplace = True)
test_data.drop('Ticket', axis = 1, inplace = True)

In [ ]:
train_data.columns

In [ ]:
## Also PassengerID is not significant we can drop it
train_data.drop('PassengerId', axis = 1, inplace = True)
test_data.drop('PassengerId', axis = 1, inplace = True)

In [ ]:
train_data.columns

In [ ]:
train_data.Name.head()

In [ ]:
## We will use the Name feature to extract the Titles from the Name..That might be helpful in predicting Survived
## We will extract it using Series.str.extract 

In [ ]:
train_data['Name'] =  train_data['Name'].str.extract(pat = "(Mr|Master|Mrs|Miss|Major|Rev|Lady|Dr|Mme|Mlle|Col|Capt)\\.")

In [ ]:
test_data['Name'] =  test_data['Name'].str.extract(pat = "(Mr|Master|Mrs|Miss|Major|Rev|Lady|Dr|Mme|Mlle|Col|Capt)\\.")

In [ ]:
train_data.Name.describe()

In [ ]:
test_data.Name.describe()

In [ ]:
train_data.Name.value_counts()

In [ ]:
test_data.Name.value_counts()

In [ ]:
train_data.Name.isnull().sum()

In [ ]:
test_data.Name.isnull().sum()

In [ ]:
## As we can see that most of the Name accounts for Miss,Mr,Mrs,Master we can group the others to Rare

In [ ]:
train_data['Name'] = train_data['Name'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr','Mme','Mlle',\
                                            'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

In [ ]:
train_data['Name'].replace(['Mr','Mrs','Miss','Master','Rare'],[1,2,3,4,5],inplace=True)

In [ ]:
train_data.Name = train_data.Name.fillna(0)

In [ ]:
train_data.Name.describe()

In [ ]:
train_data.Name = train_data.Name.astype('category')

In [ ]:
train_data.Name.describe()

In [ ]:
train_data.Name.value_counts()

In [ ]:
test_data['Name'] = test_data['Name'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr','Mme','Mlle',\
                                            'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

In [ ]:
test_data['Name'].replace(['Mr','Mrs','Miss','Master','Rare'],[1,2,3,4,5],inplace=True)

In [ ]:
test_data.Name = test_data.Name.fillna(0)

In [ ]:
test_data.Name.describe()

In [ ]:
test_data.Name = test_data.Name.astype('category')

In [ ]:
test_data.Name.describe()

In [ ]:
test_data.Name.value_counts() 

In [ ]:
## We need to convert the ‘age’ feature. First we will convert it from float into integer. 
## Then we will create the new ‘AgeGroup” variable, by categorizing every age into a group using the pandas qcut function for data binning
## The pandas documentation describes qcut as a “Quantile-based discretization function.” This basically means that qcut tries to divide up the underlying data into equal sized bins. 
## The function defines the bins using percentiles based on the distribution of the data, not the actual numeric edges of the bins.

In [ ]:
train_data.Age.describe()

In [ ]:
train_data.Age = train_data.Age.astype(int) 

In [ ]:
test_data.Age = test_data.Age.astype(int) 

In [ ]:
train_data.Age.describe()

In [ ]:
train_data['Age_new'] = pd.qcut(train_data['Age'], q = 6,duplicates = 'drop')

In [ ]:
train_data.Age_new.value_counts()

In [ ]:
train_data.head()

In [ ]:
## Now we need to convert the Age to category
le1 = LabelEncoder()
train_data.Age_new = le1.fit_transform(train_data.Age_new)
train_data.Age_new = train_data.Age_new.astype('category')

In [ ]:
train_data.Age_new.value_counts()

In [ ]:
test_data['Age_new'] = pd.qcut(test_data['Age'], q = 6, duplicates = 'drop')

In [ ]:
test_data.Age_new.value_counts()

In [ ]:
test_data.Age_new = le1.fit_transform(test_data.Age_new)
test_data.Age_new = test_data.Age_new.astype('category')

In [ ]:
test_data.Age_new.value_counts()

In [ ]:
## For the ‘Fare’ feature, we need to do the same as with the ‘Age’ feature

In [ ]:
train_data.Fare.describe()
## Min value is 0 Max is 512 
## We need to categorize Fare value with cut

In [ ]:
train_data['New_Fare'] = pd.qcut(train_data['Fare'], q = 6)

In [ ]:
train_data.New_Fare.value_counts()

In [ ]:
train_data.New_Fare = le1.fit_transform(train_data.New_Fare)
train_data.New_Fare = train_data.New_Fare.astype('category')

In [ ]:
train_data.New_Fare.value_counts()

In [ ]:
test_data['New_Fare'] = pd.qcut(test_data['Fare'], q = 6)

In [ ]:
test_data.New_Fare.value_counts()

In [ ]:
test_data.New_Fare = le1.fit_transform(test_data.New_Fare)
test_data.New_Fare = test_data.New_Fare.astype('category')

In [ ]:
test_data.New_Fare.value_counts()

In [ ]:
train_data.head()

In [ ]:
## Drop the columns which are not significant

In [ ]:
train_data.drop('Age', axis = 1, inplace = True)
train_data.drop('Fare', axis = 1, inplace = True)
test_data.drop('Age', axis = 1, inplace = True)
test_data.drop('Fare', axis = 1, inplace = True)
train_data.drop('not_alone', axis = 1, inplace = True)
test_data.drop('not_alone', axis = 1, inplace = True)

In [ ]:
train_data.columns

In [ ]:
test_data.columns